1. (전체)pdf 파일을 csv로 만들기

In [ ]:
import pdfplumber
import pandas as pd
import re

# PDF 파일 경로
pdf_file_path = r"./libs/pdf/3-1. 강의시간표 전체(2024-2).pdf"

# 저장할 CSV 파일 경로 .\libs\csv
csv_file_path = ".\libs\csv/kwu-24-2-timetable.csv"

# 데이터를 저장할 리스트 초기화
data = []

# PDF 파일 열기
with pdfplumber.open(pdf_file_path) as pdf:
    # 각 페이지 순회
    for page in pdf.pages:
        # 페이지에서 표 추출
        table = page.extract_table()
        if table:
            # 추출된 표 데이터를 리스트에 추가
            data.extend(table)

# 데이터가 없는 빈 행을 필터링
filtered_data = [row for row in data if any(cell.strip() for cell in row)]

# 추출된 데이터를 DataFrame으로 변환
df = pd.DataFrame(filtered_data[1:], columns=filtered_data[0])  # 첫 번째 행을 헤더로 사용

# DataFrame을 CSV 파일로 저장
df.to_csv(csv_file_path, index=False)

print("CSV 파일이 저장되었습니다.")


# 저장할 CSV 파일 경로 .\libs\csv
# csv_file_path = ".\libs\csv/kwu-24-2-timetable.csv"

2. 해당 csv 파일에서 학정번호형식을 제외한 '비고', '헤더피쳐'가 중복되고 필요없는 정보임으로 삭제 

In [ ]:
import re

# 학정번호 형식을 정규 표현식으로 정의 (예: I040-2-6387-02)
pattern = r'^[A-Za-z0-9]+-\d-\d{4}-\d{2}$'

# 학정번호 형식과 일치하는 행만 남기기
df = df[df['학정번호'].apply(lambda x: bool(re.match(pattern, str(x))))]

In [ ]:
df

3. '학정번호' 칼럼을 기준으로 중복값 출력 및 처리 (매치업 강의는 전선을 남기고 교선을 없애기)

In [ ]:
# '학정번호' 기준으로 중복 행을 찾아 처리
def resolve_duplicates(group):
    # '전선'이 있는 경우 그 행만 남기고 나머지 제거
    if '전선' in group['이수'].values:
        return group[group['이수'] == '전선']
    return group

df = df.groupby('학정번호', group_keys=False).apply(resolve_duplicates)

In [13]:
# import pandas as pd
# import re

# # CSV 파일 이름 목록
# csvs = ["3-1. 강의시간표 전체(2024-2)", "3-2. 교양,매치업,군사학", "3-3. 연계,창업,융합,kw-vip", "3-4. 전공(전체)"]

# # 전처리된 데이터프레임을 저장할 리스트
# dataframes = []

# for i in range(len(csvs)):
#     # CSV 파일 읽어오기
#     df = pd.read_csv(csvs[i] + '.csv', encoding='euc-kr')

#     # 0번 인덱스 행 삭제
#     df = df.drop(index=0)

#     # 1번 인덱스 행을 칼럼명으로 설정
#     df.columns = df.iloc[0]
    
#     # 1번 인덱스 행 삭제
#     df = df.drop(index=1)

#     # 학정번호 형식을 정규 표현식으로 정의 (예: I040-2-6387-02)
#     pattern = r'^[A-Za-z0-9]+-\d-\d{4}-\d{2}$'

#     # 학정번호 형식과 일치하는 행만 남기기
#     df = df[df['학정번호'].apply(lambda x: bool(re.match(pattern, str(x))))]

#     # 전처리된 데이터프레임을 리스트에 추가
#     dataframes.append(df)

# # 모든 데이터프레임을 하나로 합치기
# merged_df = pd.concat(dataframes, ignore_index=True)

# # 합쳐진 데이터프레임을 CSV 파일로 저장 (선택적)
# merged_df.to_csv('강의_전체.csv', index=False, encoding='euc-kr')

# # 최종 데이터프레임 출력
# print(merged_df.head())


1            학정번호    과목명                           분반   연계  이수 학점 시수 담당교수  \
0  0000-1-8128-01  광운인되기                   외국인학생 수강불가  NaN  교필  1  1  김정권   
1  0000-1-8128-02  광운인되기                    외국인만 수강가능  NaN  교선  1  1  한재현   
2  I000-1-3362-01   대학영어  인공지능 융합대학 공통\n(C1~C9,J5~J8)  NaN  교필  3  3  이진영   
3  I000-1-3362-02   대학영어                          NaN  NaN  교필  3  3  손영희   
4  I000-1-3362-03   대학영어                          NaN  NaN  교필  3  3  오경애   

1 강의시간 (요일,교시)                        강의유형 연계전공  
0          NaN  원격수업100,융합강의 1학기 또는 2학기 이수  NaN  
1           금2                       외국인전용  NaN  
2        월1,수2                         NaN  NaN  
3        월2,수1                         NaN  NaN  
4        월3,수4                         NaN  NaN  


In [11]:
# import pandas as pd
# import re

# csvs = ["3-1. 강의시간표 전체(2024-2)","3-2. 교양,매치업,군사학", "3-3. 연계,창업,융합,kw-vip", "3-4. 전공(전체)"]

# for i in range(len(csvs)):
#     # CSV 파일 읽어오기
#     df = pd.read_csv(csvs[i]+'.csv', encoding='euc-kr')


#     # 0번 인덱스 행 삭제
#     df = df.drop(index=0)

#     # 1번 인덱스 행을 칼럼명으로 설정
#     df.columns = df.iloc[0]
#     # 0번 인덱스 행 삭제
#     df = df.drop(index=1)
#     # 상위 3-4개 행 출력하기
#     print(df.head(4))


#     # 학정번호 형식을 정규 표현식으로 정의 (예: I040-2-6387-02)
#     pattern = r'^[A-Za-z0-9]+-\d-\d{4}-\d{2}$'

#     # 학정번호 형식과 일치하지 않는 행을 필터링
#     invalid_rows = df[~df['학정번호'].apply(lambda x: bool(re.match(pattern, str(x))))]

#     # 학정번호 형식과 일치하는 행만 남기기
#     df = df[df['학정번호'].apply(lambda x: bool(re.match(pattern, str(x))))]

#     # 삭제된 행들을 CSV 파일로 저장
#     invalid_rows.to_csv('중간과정/1.invalid_rows'+csvs[i]+'.csv', index=False, encoding='euc-kr')

#     # 수정된 데이터프레임 출력
#     print(df.head())

#     # 학정번호 형식과 일치하지 않는 행들을 확인하기 위해 출력 (선택적)
#     print(invalid_rows.head())


1            학정번호    과목명                           분반   연계  이수 학점 시수 담당교수  \
2  0000-1-8128-01  광운인되기                   외국인학생 수강불가  NaN  교필  1  1  김정권   
3  0000-1-8128-02  광운인되기                    외국인만 수강가능  NaN  교선  1  1  한재현   
4  I000-1-3362-01   대학영어  인공지능 융합대학 공통\n(C1~C9,J5~J8)  NaN  교필  3  3  이진영   
5  I000-1-3362-02   대학영어                          NaN  NaN  교필  3  3  손영희   

1 강의시간 (요일,교시)                        강의유형  
2          NaN  원격수업100,융합강의 1학기 또는 2학기 이수  
3           금2                       외국인전용  
4        월1,수2                         NaN  
5        월2,수1                         NaN  
1            학정번호    과목명                           분반   연계  이수 학점 시수 담당교수  \
2  0000-1-8128-01  광운인되기                   외국인학생 수강불가  NaN  교필  1  1  김정권   
3  0000-1-8128-02  광운인되기                    외국인만 수강가능  NaN  교선  1  1  한재현   
4  I000-1-3362-01   대학영어  인공지능 융합대학 공통\n(C1~C9,J5~J8)  NaN  교필  3  3  이진영   
5  I000-1-3362-02   대학영어                          NaN  NaN  교필  3  3  손영희   
6  I

### 4. 분반에 대한 단어 포함관계 확인

분반에서 추출할 수 있는 정보는 
1. 학과별 분반 정보 ex)"공대합반", "인융대(A1,A2)", "V1,V4", "J1~J3", 참빛인재대학 전용, '타학과생(다전공생 포함)\n수강불가'
2. 외국인 ex)외국인만 수강가능 , 외국인 수강 불가
3. 학년정보 1학년 2학년 3학년
4. 체육특기자만 수강 가능 여부
5. 재수강용

In [1]:
import pandas as pd

# CSV 파일 읽어오기
df = pd.read_csv('강의_전체.csv', encoding='euc-kr')


# '체육특기'가 포함된 행 필터링, NaN 값을 False로 처리
foreign_students = df[df['분반'].str.contains('체육특기', na=False)]

# '외국인'이 포함된 유니크한 값 추출
unique_foreign_classes = foreign_students['분반'].unique()

# 결과 출력
print("분반에 포함된 유니크한 값:")
print(unique_foreign_classes)


분반에 포함된 유니크한 값:
['체육특기자만 수강가능']


In [2]:
# '체육특기'가 포함된 행 필터링, NaN 값을 False로 처리
foreign_students = df[df['분반'].str.contains('외국인', na=False)]

# '외국인'이 포함된 유니크한 값 추출
unique_foreign_classes = foreign_students['분반'].unique()

# 결과 출력
print("분반에 포함된 유니크한 값:")
print(unique_foreign_classes)


분반에 포함된 유니크한 값:
['외국인학생 수강불가' '외국인만 수강가능' '공대 합반\n외국인만수강가능' '전정대 합반 외국인만 수강가능']


In [3]:
# '체육특기'가 포함된 행 필터링, NaN 값을 False로 처리
foreign_students = df[df['분반'].str.contains('참빛', na=False)]

# '외국인'이 포함된 유니크한 값 추출
unique_foreign_classes = foreign_students['분반'].unique()

# 결과 출력
print("분반에 포함된 유니크한 값:")
print(unique_foreign_classes)


분반에 포함된 유니크한 값:
['참빛인재대학 전용']


In [4]:
# '체육특기'가 포함된 행 필터링, NaN 값을 False로 처리
foreign_students = df[df['분반'].str.contains('타학과', na=False)]

# '외국인'이 포함된 유니크한 값 추출
unique_foreign_classes = foreign_students['분반'].unique()

# 결과 출력
print("분반에 포함된 유니크한 값:")
print(unique_foreign_classes)


분반에 포함된 유니크한 값:
['타학과생(다전공생 포함)\n수강불가']


In [5]:
# '체육특기'가 포함된 행 필터링, NaN 값을 False로 처리
foreign_students = df[df['분반'].str.contains('학년', na=False)]

# '외국인'이 포함된 유니크한 값 추출
unique_foreign_classes = foreign_students['분반'].unique()

# 결과 출력
print("분반에 포함된 유니크한 값:")
print(unique_foreign_classes)


분반에 포함된 유니크한 값:
['로봇 2학년 택 1' '1학년만 수강가능' '2~4학년 수강대상' '전정대 1학년' '인융대 1학년' '공대 1학년'
 '자연대 1학년' '전자 2학년' '전통 2학년' '전융 2학년' '화공 2학년' '건축공학 2학년' '소프트 2학년'
 '2~4학년 수강분반']


In [6]:
# '체육특기'가 포함된 행 필터링, NaN 값을 False로 처리
foreign_students = df[df['분반'].str.contains('재수강', na=False)]

# '외국인'이 포함된 유니크한 값 추출
unique_foreign_classes = foreign_students['분반'].unique()

# 결과 출력
print("분반에 포함된 유니크한 값:")
print(unique_foreign_classes)

분반에 포함된 유니크한 값:
['재수강용']


In [12]:
import pandas as pd
import re
# CSV 파일 읽어오기
df = pd.read_csv('강의_전체.csv', encoding='euc-kr')

# NaN 값을 빈 문자열로 대체
df['분반'] = df['분반'].fillna("")
# NaN 값을 빈 문자열로 대체
df['분반_원본'] = df['분반'].fillna("")


In [13]:
# 조건에 따라 열 값 수정
conditions_foreign = ['외국인만 수강가능', '공대 합반\n외국인만수강가능', '전정대 합반 외국인만 수강가능']
condition_domestic = '외국인학생 수강불가'


# 외국인재학생 = True, 국내재학생 = False 설정
for condition in conditions_foreign:
    df.loc[df['분반'] == condition, ['외국인재학생', '국내재학생']] = [True, False]


# 외국인재학생 = False, 국내재학생 = True 설정
df.loc[df['분반'] == condition_domestic, ['외국인재학생', '국내재학생']] = [False, True]

C:\Users\Admin\AppData\Local\Temp\ipykernel_20700\3695925010.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df['분반'] == condition, ['외국인재학생', '국내재학생']] = [True, False]
C:\Users\Admin\AppData\Local\Temp\ipykernel_20700\3695925010.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df['분반'] == condition, ['외국인재학생', '국내재학생']] = [True, False]


## 칼럼 추가

In [14]:
# '참빛인재대학'이 포함되어 있는지 확인하고, 조건에 따라 새 열에 'V1,V2,V3,V4' 할당
df['분반_학과1'] = df['분반'].apply(lambda x: 'V1,V2,V3,V4' if '참빛인재대학' in x else "")

# '참빛인재대학'이 포함되어 있는지 확인하고, 조건에 따라 새 열에 'TRUE' 할당
df['체육특기자'] = df['분반'].apply(lambda x: 'TRUE' if '체육특기자' in x else "")

# '참빛인재대학'이 포함되어 있는지 확인하고, 조건에 따라 새 열에 'TRUE' 할당
df['재수강용'] = df['분반'].apply(lambda x: 'TRUE' if '재수강' in x else "")

# '참빛인재대학'이 포함되어 있는지 확인하고, 조건에 따라 새 열에 'TRUE' 할당
df['타학과수강불가'] = df['분반'].apply(lambda x: 'FALSE' if '타학과' in x else "")

In [15]:
# 삭제할 문자열 리스트
remove_items = ['체육실기', '음악실기', '미술실기']

# 특정 문자열 삭제
for item in remove_items:
    df['분반'] = df['분반'].str.replace(item, '')


## 학년 처리

In [23]:
# 새로운 열 추가를 위한 함수 정의
def extract_grade_info(row):
    grades = []
    if '1학년' in row:
        grades.append('1')
    if '2학년' in row:
        grades.append('2')
    if '3학년' in row:
        grades.append('3')
    if '2~4학년' in row:
        grades.append('2,3,4')
    return ','.join(grades)

# 새로운 열 '학년 정보' 추가
df['학년'] = df['분반'].apply(extract_grade_info)
df['학년'].value_counts()

KeyError: '분반'

## 분반 처리

In [17]:

# NaN 값을 빈 문자열로 대체
df['분반'] = df['분반'].fillna("")

#A1~A2이런식으로 생긴 애들 모음
def expand_ranges(text):
    # 정규 표현식을 사용하여 패턴을 인식하고 대체
    pattern = re.compile(r'([A-Z])(\d+)~\1(\d+)')

    # 각 패턴에 대해 확장된 범위로 대체하고, 그 결과만 반환
    def replace_range(match):
        start_char = match.group(1)
        start_num = int(match.group(2))
        end_num = int(match.group(3))
        return ','.join(f"{start_char}{i}" for i in range(start_num, end_num + 1))
    
    # 전체 문자열 대신, 일치하는 부분만 찾아서 리스트로 모아 반환
    expanded_parts = pattern.findall(text)
    expanded_results = [replace_range(match) for match in pattern.finditer(text)]
    
    return ','.join(expanded_results)  # 리스트를 문자열로 결합
def replace_spaces_with_commas(text):
    # 모든 공백을 콤마로 대체
    return text.replace(" ", ",")

# '분반' 열에 함수 적용
df['분반_학과'] = df['분반'].apply(expand_ranges)

# 결과 출력
print(df)

                학정번호        과목명                           분반   연계  이수  학점  시수  \
0     0000-1-8128-01      광운인되기                   외국인학생 수강불가  NaN  교필   1   1   
1     0000-1-8128-02      광운인되기                    외국인만 수강가능  NaN  교선   1   1   
2     I000-1-3362-01       대학영어  인공지능 융합대학 공통\n(C1~C9,J5~J8)  NaN  교필   3   3   
3     I000-1-3362-02       대학영어                               NaN  교필   3   3   
4     I000-1-3362-03       대학영어                               NaN  교필   3   3   
...              ...        ...                          ...  ...  ..  ..  ..   
3111  J030-1-7714-01    콘텐츠기획개론                           V2  NaN  전필   3   3   
3112  J040-1-0444-01      디지털공학                           V3  NaN  전필   3   3   
3113  J040-1-1186-01       이산수학                           V3  NaN  전선   3   3   
3114  J050-1-5003-01       건강교육                           V4  NaN  전선   3   3   
3115  J050-1-5004-01  스포츠엔터테인먼트                           V4  NaN  전선   3   3   

     담당교수 강의시간 (요일,교시)     

In [18]:
def remove_range_patterns(text):
    # 정규 표현식을 사용하여 패턴을 인식하고 삭제
    pattern = re.compile(r'([A-Z])(\d+)~\1(\d+)')
    
    # 패턴과 일치하는 부분을 빈 문자열로 대체
    result = pattern.sub('', str(text))
    
    # 결과 문자열의 앞뒤 공백 제거
    return result.strip()

# '분반' 열에 함수 적용
df['분반'] = df['분반'].apply(remove_range_patterns)


In [19]:
#A1,A2이런식으로 생긴 애들 모음

def separate_alphanumeric_patterns(text):
    # 알파벳 하나와 숫자로 이루어진 패턴을 찾는 정규 표현식
    pattern = re.compile(r'([A-Z]\d+)')
    
    # 패턴과 일치하는 모든 부분 찾기
    matches = pattern.findall(str(text))
    
    # 패턴과 일치하는 부분을 제거한 나머지 텍스트
    remaining_text = pattern.sub('', str(text)).strip()
    
    # 찾은 패턴들을 콤마로 구분하여 하나의 문자열로 만듦
    patterns = ','.join(matches) if matches else ''
    
    return remaining_text, patterns


# '분반' 열에 함수 적용
df['분반_정리'], df['분반_패턴'] = zip(*df['분반'].apply(separate_alphanumeric_patterns))

# 결과 출력
print(df)

                학정번호        과목명                 분반   연계  이수  학점  시수 담당교수  \
0     0000-1-8128-01      광운인되기         외국인학생 수강불가  NaN  교필   1   1  김정권   
1     0000-1-8128-02      광운인되기          외국인만 수강가능  NaN  교선   1   1  한재현   
2     I000-1-3362-01       대학영어  인공지능 융합대학 공통\n(,)  NaN  교필   3   3  이진영   
3     I000-1-3362-02       대학영어                     NaN  교필   3   3  손영희   
4     I000-1-3362-03       대학영어                     NaN  교필   3   3  오경애   
...              ...        ...                ...  ...  ..  ..  ..  ...   
3111  J030-1-7714-01    콘텐츠기획개론                 V2  NaN  전필   3   3  정형원   
3112  J040-1-0444-01      디지털공학                 V3  NaN  전필   3   3  손광철   
3113  J040-1-1186-01       이산수학                 V3  NaN  전선   3   3  황이환   
3114  J050-1-5003-01       건강교육                 V4  NaN  전선   3   3   박석   
3115  J050-1-5004-01  스포츠엔터테인먼트                 V4  NaN  전선   3   3  김준희   

     강의시간 (요일,교시)                        강의유형  ... 외국인재학생  국내재학생 분반_학과1 체육특기자  \
0     

In [20]:
def combine_and_deduplicate(row):
    # '분반_학과'와 '분반_패턴'의 값을 합칩니다
    combined = f"{row['분반_학과']},{row['분반_학과1']},{row['분반_패턴']}"
    
    # 콤마로 분리하고 공백을 제거합니다
    patterns = [p.strip() for p in combined.split(',') if p.strip()]
    
    # 중복을 제거하고 정렬합니다
    unique_patterns = sorted(set(patterns))
    
    # 다시 콤마로 구분된 문자열로 만듭니다
    return ','.join(unique_patterns)

# '분반_최종' 열 생성
df['분반_최종'] = df.apply(combine_and_deduplicate, axis=1)

# 결과 출력
print(df[['분반_학과', '분반_패턴','분반_학과1', '분반_최종']])

# 불필요한 열 삭제
columns_to_drop = ['분반_학과', '분반_학과1', '분반_정리', '분반_패턴', '분반']
df = df.drop(columns=columns_to_drop)

                                       분반_학과 분반_패턴 분반_학과1  \
0                                                           
1                                                           
2     C1,C2,C3,C4,C5,C6,C7,C8,C9,J5,J6,J7,J8                
3                                                           
4                                                           
...                                      ...   ...    ...   
3111                                            V2          
3112                                            V3          
3113                                            V3          
3114                                            V4          
3115                                            V4          

                                       분반_최종  
0                                             
1                                             
2     C1,C2,C3,C4,C5,C6,C7,C8,C9,J5,J6,J7,J8  
3                                             
4                               

In [21]:
#타학과생(다전공생 포함)\n수강불가 -> 수동으로 분반_최종에 추가하는 처리 필요

In [22]:
# 결과를 CSV 파일로 저장
df.to_csv('df_result_expanded2.csv', index=False, encoding='euc-kr')

## 추가로 해야할 일

1. 전공인정분반 추가
2. 타학과전공인정과목분반 추가
3. 선수과목 관계 추가

<윤나>

1. 학정번호 기준 중복값 처리 (매치업 강의는 전선을 남기기)
2. ‘분반_최종’을 ‘수강가능한 학과’로 이름을 바꾸고, 나머지 빈 셀을 모든 학과 코드 적기
3. ‘전공 인정 학과’ 새 칼럼을 생성하고, ‘이수’칼럼이 ‘전선’, ‘전필’인 경우에 해당하는 칼럼으로,
학정번호 앞 4글자가 아래 표에서 찾아서 해당 학과코드를 넣기
4. 강의시간 좌표형 변환
5. 강의 테마 칼럼 추가
’이수’가 교선,교필일 경우에 해당하는 칼럼으로 → 코드대신 엑셀에서 드래그가 훨씬 나을거 같음

In [2]:
import pandas as pd

df1 = pd.read_csv(r"./강의계획서_전체_v1.csv", encoding="cp949")

In [3]:
df1.columns

Index(['학정번호', '과목명', '연계', '이수', '학점', '시수', '담당교수', '강의시간 (요일,교시)', '강의유형',
       '연계전공', '분반_원본', '외국인재학생', '국내재학생', '체육특기자', '재수강용', '타학과수강불가', '학년',
       '분반_최종'],
      dtype='object')